In [1]:
import requests
import pandas as pd
import os 
import numpy as np
from sqlalchemy import create_engine

In [11]:
#Definition call all the APIs needed 

def retrieve(GET):
    #URL to get API with name to get specific table
    url = f"https://pokemon-go1.p.rapidapi.com/{GET}.json"
    #Dictionary that holds the API host and Key
    headers = {
    'x-rapidapi-key': "76d2396840mshc562c26618d33a2p1fb1e6jsn95b66aa3ea3c",
    'x-rapidapi-host': "pokemon-go1.p.rapidapi.com"
    }
    #putting together the URL and Dictionary(API keys) to pull from API
    response = requests.request("GET", url, headers=headers).json()
    #create a pandas DF out of the API Call
    df = pd.json_normalize(response)
#     return DataFrame
    return df

In [12]:
# Choosen APIs I want to have called. 
GETS = ['pokemon_stats','pokemon_max_cp','fast_moves','charged_moves','pokemon_encounter_data','pokemon_types']

# Create a dictionary that will hold the the API Calls
RapidAPI_dic = {}

# Run through the definition with all of the APIs that I chose and store them in the dictionary
for i in GETS:
    RapidAPI_dic[i] = retrieve(i)



In [13]:
retrieve("pokemon_stats")

,base_attack,base_defense,base_stamina,form,pokemon_id,pokemon_name
0,118,111,128,Fall_2019,1,Bulbasaur
1,118,111,128,Normal,1,Bulbasaur
2,118,111,128,Purified,1,Bulbasaur
3,118,111,128,Shadow,1,Bulbasaur
4,151,143,155,Normal,2,Ivysaur
...,...,...,...,...,...,...
1278,212,179,190,Purified,866,Mr. Rime
1279,212,179,190,Shadow,866,Mr. Rime
1280,163,237,151,Galarian,867,Runerigus
1281,163,237,151,Purified,867,Runerigus


In [4]:
#as seen when this Data Frame is run there are multiple forms of the same pokemon.
Rapid_Stats = RapidAPI_dic['pokemon_stats']
Rapid_Stats

,base_attack,base_defense,base_stamina,form,pokemon_id,pokemon_name
0,118,111,128,Fall_2019,1,Bulbasaur
1,118,111,128,Normal,1,Bulbasaur
2,118,111,128,Purified,1,Bulbasaur
3,118,111,128,Shadow,1,Bulbasaur
4,151,143,155,Normal,2,Ivysaur
...,...,...,...,...,...,...
1117,195,162,172,Purified,863,Perrserker
1118,195,162,172,Shadow,863,Perrserker
1119,248,176,158,Normal,865,Sirfetch’d
1120,248,176,158,Purified,865,Sirfetch’d


In [6]:
# First we must see if any of the pokemon stats vary over the different forms.
# create a total stats column to make comparison against each form easier
Rapid_Stats['total_stats'] = Rapid_Stats['base_defense'] + Rapid_Stats['base_attack'] 

#Take all pokemon with Normal Forms and put them in a DataFrame
Normal = Rapid_Stats['form'] == 'Normal'
Normal = Rapid_Stats[Normal]
Normal
# Create variable to hold total stat number
stats = 0
# 
counter = 1
#create a for loop to check each pokemon form against itself
for index, row in Rapid_Stats.iterrows():
    if row[5] in Normal.values:
        stats = Normal.loc[Normal["pokemon_name"]==row[5]]['total_stats'].values[0]
        if stats != row[6]:
            
            newindex = Normal[-1:].index[0]+ 1
            Normal.loc[index, 'base_attack'] = int(row[0])
            Normal.loc[index,'base_defense'] = int(row[1])
            Normal.loc[index,'base_stamina'] = int(row[2])
            Normal.loc[index,'form'] = row[3]
            Normal.loc[index,'pokemon_id'] = row[4]
            Normal.loc[index,'pokemon_name'] = row[5]
            Normal.loc[index,'total_stats'] = int(row[6])
#             print(newindex)
            counter += 1
    elif row[5] not in Normal.values:
        newindex = Normal[-1:].index[0]+ 1
        Normal.loc[index, 'base_attack'] = int(row[0])
        Normal.loc[index,'base_defense'] = int(row[1])
        Normal.loc[index,'base_stamina'] = int(row[2])
        Normal.loc[index,'form'] = row[3]
        Normal.loc[index,'pokemon_id'] = row[4]
        Normal.loc[index,'pokemon_name'] = row[5]
        Normal.loc[index,'total_stats'] = int(row[6])
#         print(f'{newindex} Not Normal')
        counter += 1
        
print(counter)

C:\Users\Lenny Ovo\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


51


In [7]:
# Sort Values by index and check table
Normal = Normal.sort_index()
Normal

,base_attack,base_defense,base_stamina,form,pokemon_id,pokemon_name,total_stats
1,118.0,111.0,128.0,Normal,1.0,Bulbasaur,229.0
4,151.0,143.0,155.0,Normal,2.0,Ivysaur,294.0
8,198.0,189.0,190.0,Normal,3.0,Venusaur,387.0
12,116.0,93.0,118.0,Normal,4.0,Charmander,209.0
15,158.0,126.0,151.0,Normal,5.0,Charmeleon,284.0
...,...,...,...,...,...,...,...
1111,118.0,99.0,130.0,Normal,808.0,Meltan,217.0
1112,226.0,190.0,264.0,Normal,809.0,Melmetal,416.0
1113,180.0,194.0,212.0,Galarian,862.0,Obstagoon,374.0
1116,195.0,162.0,172.0,Galarian,863.0,Perrserker,357.0


In [8]:
# Adding Max CP to the Data Frame
Rapid_CP = RapidAPI_dic['pokemon_max_cp']
#Remove unneeded columns
Rapid_CP = Rapid_CP.drop(['form' , 'pokemon_id' , 'pokemon_name'] , axis = 1)
# Join tables 
Rapid = pd.concat([Normal , Rapid_CP], axis = 1 , join = 'inner')


#Pulling in types of each pokemon
Rapid_Types = RapidAPI_dic['pokemon_types']
# there is two types inside of the type column, it has to be separated into two seperate columns
types =  pd.DataFrame(Rapid_Types['type'].tolist())
# Join main table and types table
Rapid_Final = pd.concat([ Rapid , types], axis = 1 , join = 'inner')
# Rename columns
Rapid_Final = Rapid_Final.rename(columns={ 0: "primary_type", 1: "secondary_type" , 'base_attack' : 'attack' , 
                           'base_defense' : 'defense' , 'base_stamina' : 'stamina' , 'pokemon_id': 'pokemon_id' , 
                           'total_stats' : 'total_stats' , 'max_cp' : 'max_cp' , 'form': 'form' ,'pokemon_name' : 'name'})
# Reorganize tables
Rapid_Final = Rapid_Final[['name' , 'pokemon_id' , 'primary_type' , 'secondary_type', 'attack' , 'defense' , 'stamina' , 'total_stats' , 'max_cp' , 'form']]
# check table
Rapid_Final

,name,pokemon_id,primary_type,secondary_type,attack,defense,stamina,total_stats,max_cp,form
1,Bulbasaur,1.0,Grass,Poison,118.0,111.0,128.0,229.0,1115,Normal
4,Ivysaur,2.0,Grass,Poison,151.0,143.0,155.0,294.0,1699,Normal
8,Venusaur,3.0,Grass,Poison,198.0,189.0,190.0,387.0,2720,Normal
12,Charmander,4.0,Fire,None,116.0,93.0,118.0,209.0,980,Normal
15,Charmeleon,5.0,Fire,None,158.0,126.0,151.0,284.0,1653,Normal
...,...,...,...,...,...,...,...,...,...,...
1111,Meltan,808.0,Steel,None,118.0,99.0,130.0,217.0,1068,Normal
1112,Melmetal,809.0,Steel,None,226.0,190.0,264.0,416.0,3599,Normal
1113,Obstagoon,862.0,Dark,Normal,180.0,194.0,212.0,374.0,2652,Galarian
1116,Perrserker,863.0,Steel,None,195.0,162.0,172.0,357.0,2386,Galarian


In [9]:
# Pull encounter data from list of Dataftames drop unneded columns and present table
Rapid_encounter = RapidAPI_dic['pokemon_encounter_data']
Rapid_encounter = Rapid_encounter.drop(['form' , 'pokemon_id' , 'pokemon_name'] , axis = 1)
Rapid_encounter

,attack_probability,base_capture_rate,base_flee_rate,dodge_probability,max_pokemon_action_frequency,min_pokemon_action_frequency
0,0.1,0.20,0.10,0.15,1.6,0.2
1,0.1,0.20,0.10,0.15,1.6,0.2
2,0.1,0.20,0.10,0.15,1.6,0.2
3,0.7,0.05,0.10,0.20,1.6,0.2
4,0.1,0.10,0.07,0.15,1.6,0.2
...,...,...,...,...,...,...
1117,0.3,0.10,0.07,0.45,1.6,0.2
1118,0.7,0.10,0.07,0.20,1.6,0.2
1119,0.3,0.10,0.07,0.45,1.6,0.2
1120,0.3,0.10,0.07,0.45,1.6,0.2


In [10]:
# Adding Max CP to the Data Frame
Both = pd.concat([Rapid_Final , Rapid_encounter], axis = 1 , join = 'inner')
# Reorganize table
Both = Both[["pokemon_id" , "name" , "primary_type" , "secondary_type" , "form", "max_cp" ,  "stamina" , "attack" , "defense" ,
      "total_stats" ,   "attack_probability" , "base_capture_rate" , "base_flee_rate" , "dodge_probability" ,
      "max_pokemon_action_frequency" , "min_pokemon_action_frequency" ]]
Both

In [11]:
# Split DF for SQL DF and check middle split
Rapid_Split = np.split(Both, [5,10] , axis =1)
Rapid_Split[1]

,max_cp,stamina,attack,defense,total_stats
1,1115,128,118,111,229
4,1699,155,151,143,294
8,2720,190,198,189,387
12,980,118,116,93,209
15,1653,151,158,126,284
...,...,...,...,...,...
1111,1068,130,118,99,217
1112,3599,264,226,190,416
1113,2652,212,180,194,374
1116,2386,172,195,162,357


In [12]:
# Rename split table create a primary key and drop duplicates. Then present table
Battle_R = Rapid_Split[1]
Battle_R['primary_key'] = Battle_R.index
Battle_R = Battle_R.drop_duplicates( subset =  ['max_cp' , 'stamina' , 'attack' , 'defense' , 'total_stats'])
Battle_R

,max_cp,stamina,attack,defense,total_stats,primary_key
1,1115,128,118,111,229,1
4,1699,155,151,143,294,4
8,2720,190,198,189,387,8
12,980,118,116,93,209,12
15,1653,151,158,126,284,15
...,...,...,...,...,...,...
1111,1068,130,118,99,217,1111
1112,3599,264,226,190,416,1112
1113,2652,212,180,194,374,1113
1116,2386,172,195,162,357,1116


In [13]:
# Create placeholder for foreign key
Both['battle_fk'] = 0
# Make foreign key for reference for SQL
for index, row in Both.iterrows():
    for index2, row2 in Battle_R.iterrows():
#         If the battle rows match the row drop the Foriegn key in the foriegn key column
        if row[5] == row2[0] and row[6] == row2[1] and row[7] == row2[2] and row[8] == row2[3] and row[9] == row2[4]:
            Both.loc[index, 'battle_fk'] = int(row2[5])
           

In [14]:
# Rename split table create a primary key and drop duplicates. Then present table
Catch_R = Rapid_Split[2]
Catch_R['primary_key'] = Catch_R.index
Catch_R = Catch_R.drop_duplicates( subset =  ['attack_probability' , 'base_capture_rate' , 'base_flee_rate' , 'dodge_probability' ,  
                               'max_pokemon_action_frequency' , 'min_pokemon_action_frequency'])
Catch_R

,attack_probability,base_capture_rate,base_flee_rate,dodge_probability,max_pokemon_action_frequency,min_pokemon_action_frequency,primary_key
1,0.1,0.2,0.1,0.15,1.6,0.2,1
4,0.1,0.1,0.07,0.15,1.6,0.2,4
8,0.2,0.05,0.05,0.15,1.6,0.2,8
33,0.1,0.5,0.2,0.15,1.6,0.2,33
34,0.1,0.25,0.09,0,1.6,0.2,34
...,...,...,...,...,...,...,...
1081,0.7,0.2,0.07,0.2,1.6,0.2,1081
1086,0.1,0.7,0.15,0.15,1.6,0.2,1086
1111,0.1,0.3,0,0.15,1.6,0.2,1111
1113,0.3,0.04,0.04,0.15,1.6,0.2,1113


In [15]:
# Create placeholder for foreign key
Both['catch_fk'] = 0

# Make foreign key for reference for SQL
for index, row in Both.iterrows():
    for index2, row2 in Catch_R.iterrows():
        #         If the catch rows match the row drop the Foriegn key in the foriegn key column
        if row[10] == row2[0] and row[11] == row2[1] and row[12] == row2[2] and row[13] == row2[3] and row[14] == row2[4] and row[15] == row2[5]:
            Both.loc[index, 'catch_fk'] = int(row2[6])
    

In [16]:
# Create a primary key for the name table, make sure the pokemon id is an integer and pick the columns wanted 
Both['pk'] = Both.index
Both['pokemon_id'] = Both['pokemon_id'].astype(int)
Name_R = Both[["pokemon_id" , "name" , "primary_type" , "secondary_type" , "form" , 'battle_fk' , 'catch_fk' , 'pk']]
Name_R

,pokemon_id,name,primary_type,secondary_type,form,battle_fk,catch_fk,pk
1,1,Bulbasaur,Grass,Poison,Normal,1,1,1
4,2,Ivysaur,Grass,Poison,Normal,4,4,4
8,3,Venusaur,Grass,Poison,Normal,8,8,8
12,4,Charmander,Fire,None,Normal,12,1,12
15,5,Charmeleon,Fire,None,Normal,15,4,15
...,...,...,...,...,...,...,...,...
1111,808,Meltan,Steel,None,Normal,1111,1111,1111
1112,809,Melmetal,Steel,None,Normal,1112,1111,1112
1113,862,Obstagoon,Dark,Normal,Galarian,1113,1113,1113
1116,863,Perrserker,Steel,None,Galarian,1116,1116,1116


In [17]:
Catch_R

,attack_probability,base_capture_rate,base_flee_rate,dodge_probability,max_pokemon_action_frequency,min_pokemon_action_frequency,primary_key
1,0.1,0.2,0.1,0.15,1.6,0.2,1
4,0.1,0.1,0.07,0.15,1.6,0.2,4
8,0.2,0.05,0.05,0.15,1.6,0.2,8
33,0.1,0.5,0.2,0.15,1.6,0.2,33
34,0.1,0.25,0.09,0,1.6,0.2,34
...,...,...,...,...,...,...,...
1081,0.7,0.2,0.07,0.2,1.6,0.2,1081
1086,0.1,0.7,0.15,0.15,1.6,0.2,1086
1111,0.1,0.3,0,0.15,1.6,0.2,1111
1113,0.3,0.04,0.04,0.15,1.6,0.2,1113


In [18]:
Battle_R

,max_cp,stamina,attack,defense,total_stats,primary_key
1,1115,128,118,111,229,1
4,1699,155,151,143,294,4
8,2720,190,198,189,387,8
12,980,118,116,93,209,12
15,1653,151,158,126,284,15
...,...,...,...,...,...,...
1111,1068,130,118,99,217,1111
1112,3599,264,226,190,416,1112
1113,2652,212,180,194,374,1113
1116,2386,172,195,162,357,1116


In [19]:
# Create Connection with ash user and create engine
rds_connection_string = "ash:catchem@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [20]:
# Check tables
engine.table_names()

['rapid_catch',
 'rapid_name',
 'kaggle_catch',
 'kaggle_name',
 'kaggle_battle',
 'rapid_battle']

In [21]:
# push Catch data to catch table on DB for rapid tables
Catch_R.to_sql(name='rapid_catch', con=engine, if_exists='append', index=False)

In [22]:
# push battle data to battle table on DB for rapid tables

Battle_R.to_sql(name='rapid_battle', con=engine, if_exists='append', index=False)

In [23]:
# push name data to name table on DB for rapid tables

Name_R.to_sql(name='rapid_name', con=engine, if_exists='append', index=False)

In [28]:
# Check that push was successful
pd.read_sql_query('select * from rapid_catch', con=engine).head()

,attack_probability,base_capture_rate,base_flee_rate,dodge_probability,max_pokemon_action_frequency,min_pokemon_action_frequency,primary_key
0,0.1,0.20,0.10,0.15,1.6,0.2,1
1,0.1,0.10,0.07,0.15,1.6,0.2,4
2,0.2,0.05,0.05,0.15,1.6,0.2,8
3,0.1,0.50,0.20,0.15,1.6,0.2,33
4,0.1,0.25,0.09,0.00,1.6,0.2,34


In [26]:
# Check that push was successful
pd.read_sql_query('select * from rapid_battle', con=engine).head()

,max_cp,stamina,attack,defense,total_stats,primary_key
0,1115,128,118,111,229,1
1,1699,155,151,143,294,4
2,2720,190,198,189,387,8
3,980,118,116,93,209,12
4,1653,151,158,126,284,15


In [27]:
# Check that push was successful
pd.read_sql_query('select * from rapid_name', con=engine).head()

,pokemon_id,name,primary_type,secondary_type,form,battle_fk,catch_fk,pk
0,1,Bulbasaur,Grass,Poison,Normal,1,1,1
1,2,Ivysaur,Grass,Poison,Normal,4,4,4
2,3,Venusaur,Grass,Poison,Normal,8,8,8
3,4,Charmander,Fire,None,Normal,12,1,12
4,5,Charmeleon,Fire,None,Normal,15,4,15
